<a href="https://colab.research.google.com/github/simecek/PseudoDNA_Generator/blob/master/colabs/models/Intergenomic_AWD_LSTM_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [0]:
!pip install fastai2>=0.0.11 ipywidgets matplotlib nbdev>=0.2.12 pandas scikit_learn sentencepiece

In [0]:
from fastai2.text.all import *
import pandas as pd
import torch

In [13]:
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.get_device_name(0)

(True, 1, 'Tesla P100-PCIE-16GB')

In [3]:
# Mount to your Google Drive allowing lesson files will be saved to your Drive location
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
dt = pd.read_csv("/content/drive/My Drive/data/random/intergenomic_seqs_50k.csv")
dt = dt[~dt.seq.str.contains("N")]   # just for sure

train = dt[dt.chr!="1"]
test = dt[dt.chr=="1"]

print(dt.shape, train.shape, test.shape)
dt.head()

(50000, 4) (46286, 4) (3714, 4)


,chr,start,end,seq
0,17,14239397,14239596,AACTGGGATTCACAGGAGCTTAATGGAGCACATGATGTTAAGTGAAGTGAGCCAGGCACAAAAAGACAACTACCACGTGATCTGACTTATGTGGAATGTAAAACAATTGAACTCATGGAAGCAGAGAGTAGAATGGAGGATACCAGGGGCTGGGAGGCAGGGGTTTGGGGAGACGGTGAAAGCGTTCTAAAGTGTAGTTA
1,X,90928374,90928573,GCTAGTTGTATGGTTAGCAGCAAGATATTTTTTCTCTCTGATCTTTAATTTTCATATTTAAATTTGGCTAAGAGTACTTGCCTCTTAAAACTGTGTTGCTGGTATTACCAGAGTGTGGTATAATTAAAATATATATTTGCTTTTTGTCACCAGTTTCTCACACAGTACATCAAAAGCCCTTGCAATTTTCTGAGTGATAA
2,3,104278717,104278916,GACTTTGTAGACTTGTGTGACCTGTGTGCCTCCCTCTCCCCCCAAAAAAAACAAAAAAAATAAAGGATCTTGGGAAAGACTATATAAAAGGCAAGACTCCTTTAATGGAGGGGATATGCTAGATTGCCTCCCATTATGGCCCATGCCAAAGTGTTTAAACTTAGAAAAATGGTTCCAGTTTACTTCTGGGCTTAAAAATC
3,4,187089054,187089253,ATGTTAACACCAAATCAGTCCATCCTAATTATCACTCAAAAATCAAACATTTTTTAGGGAGGCAAAAACTGTCATGAGAACTACAATTTGATTTGGAGACTATTTCACTTATACAGTTTCTTCACATGATGACCAGCCTTCTTTCTTTAGTAATGGTTATTACTATGGCCATTGCTGTTAATTCTGTGACTTATCACTTC
4,2,137742849,137743048,GCAGGAGCTCTATCTGTTTGGACTAGTTCAGCCCCATCTCTTTTGGGGTGACTCGGGTGATGCTAAGCTTCCCAGGGCCATTGTGTTCTGTCTTCTGCCTCTGACTTTTTCCCTGCTACCCACATGAGCTTCTGCTATGCTCTCTTCTTTCCTGTCCAGAAATCATGTAGTAAGATGCTTTTTGGCTGGAGACCCTGAAA


In [0]:
!rm -rf split_tok

!rm -rf split
!mkdir split
!mkdir split/train
!mkdir split/valid
!mkdir split/train/1/
!mkdir split/valid/1/

In [0]:
# splitting the file into training and test part
N = len(dt.seq)

for i, s in enumerate(train.seq):
  open("split/train/1/seq"+str(i)+".txt", 'w').writelines([s])
for i, s in enumerate(test.seq):
  open("split/valid/1/seq"+str(i)+".txt", 'w').writelines([s])

## Tokenizer

In [7]:
!wget https://raw.githubusercontent.com/simecek/PseudoDNA_Generator/master/colabs/models/genomic_tokenizer.py

--2020-06-12 00:39:38--  https://raw.githubusercontent.com/simecek/PseudoDNA_Generator/master/colabs/models/genomic_tokenizer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 586 [text/plain]
Saving to: ‘genomic_tokenizer.py’

genomic_tokenizer.p 100%[===================>]     586  --.-KB/s    in 0s      

2020-06-12 00:39:38 (34.2 MB/s) - ‘genomic_tokenizer.py’ saved [586/586]



In [0]:
from genomic_tokenizer import tkn2

## Data Loaders

In [0]:
dls_lm = TextDataLoaders.from_folder(Path("./split"), bs=128, seed=42, 
                                   is_lm=True, 
                                   tok_tfm=tkn2, seq_len=50)

In [14]:
dls_lm.show_batch()

,text,text_
0,xxbos A T G C C A C T G C A C T C C A G C C T G G C A A C A G A G T G A G A C T C C A T C T C A A A A A,A T G C C A C T G C A C T C C A G C C T G G C A A C A G A G T G A G A C T C C A T C T C A A A A A A
1,C A C T G T A A A A T G A A C A C A G A T G T A C T T T T T C A G G G A T C A G T G T C G C T C A A,A C T G T A A A A T G A A C A C A G A T G T A C T T T T T C A G G G A T C A G T G T C G C T C A A G
2,G G G A T A T T T C A T C T C C T T T T C A T G A G A A G C A C G G T G A C C G T G A A A T A A G C,G G A T A T T T C A T C T C C T T T T C A T G A G A A G C A C G G T G A C C G T G A A A T A A G C T
3,C T C A T T G G T G A A C T A C A T C A A A T G A A T C A C C G C A T A xxbos T G C T G G A T C A A A T,T C A T T G G T G A A C T A C A T C A A A T G A A T C A C C G C A T A xxbos T G C T G G A T C A A A T G
4,A A C C A C T C A C A G C T T C C T G G G A T T G G G A C A T G G A C A T C T C T G G G G A C C A A,A C C A C T C A C A G C T T C C T G G G A T T G G G A C A T G G A C A T C T C T G G G G A C C A A T
5,G A A T A G C G G A C A C A G T C A A G C A C A G G G T G C C C T T C C A C C T G G G G C A G G G G,A A T A G C G G A C A C A G T C A A G C A C A G G G T G C C C T T C C A C C T G G G G C A G G G G G
6,C C A A G T C T T T G A G A G G A A G A A T C T G C A G C A C T G C A T A T A T A T A T A T A T A T,C A A G T C T T T G A G A G G A A G A A T C T G C A G C A C T G C A T A T A T A T A T A T A T A T A
7,T T T G C C G G G C A C G G T G G C G G G A G C C T G T A G T C C C A G C T A C T C G T G A G G C T,T T G C C G G G C A C G G T G G C G G G A G C C T G T A G T C C C A G C T A C T C G T G A G G C T G
8,T T C A T C A G G G A T A T T G G T C T G T A G T T T T C xxbos C T C A A C A T A T C T A T A T A A A T,T C A T C A G G G A T A T T G G T C T G T A G T T T T C xxbos C T C A A C A T A T C T A T A T A A A T A


## Model and Learning

In [0]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, pretrained=False, 
    metrics=[accuracy, Perplexity()])

In [0]:
learn.fit_one_cycle(5, 2e-2)
learn.export("/content/drive/My Drive/DNAModels/Intergenomic/AWD_LSTM_v4.pkl")

epoch,train_loss,valid_loss,accuracy,perplexity,time
